Only focus on the limited set of logs returns variabe.

In [33]:
import os, sys
sys.path.insert(1,"../")
from analysis_utils import *

from sklearn.ensemble import RandomForestClassifier as RF

pd.set_option('display.max_columns', 999)
%load_ext autoreload
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [192]:
def getConfusionMatrix(y,y_hat):
    tn,fp,fn,tp = confusion_matrix(y,y_hat).ravel()
    print("TN",tn,"\nFP",fp,"\nFN",fn,"\nTP",tp)
    return tn,fp,fn,tp

# I. Prepare Data
We want to make sure to incorporate **SUBSAMPLING** to make help with the lopsided distribution

In [275]:
d = pd.read_pickle("../Log_processed_data/ohlcv_data_training_h_log.pkl")
print("Daily data",d.shape)

Daily data (43506, 681)


In [276]:
# Convert exchange variable to categorical
d['exchange_cat'] = d.exchange.astype('category').cat.codes

In [277]:
d.filter(regex="sum|log").columns.sort_values()

Index(['log_12h_high', 'log_12h_open', 'log_12h_volumefrom',
       'log_12h_volumeto', 'log_24h_high', 'log_24h_open',
       'log_24h_volumefrom', 'log_24h_volumeto', 'log_36h_high',
       'log_36h_open',
       ...
       'sum_24h_volumefrom', 'sum_24h_volumeto', 'sum_36h_volumefrom',
       'sum_36h_volumeto', 'sum_3h_volumefrom', 'sum_3h_volumeto',
       'sum_6h_volumefrom', 'sum_6h_volumeto', 'sum_9h_volumefrom',
       'sum_9h_volumeto'],
      dtype='object', length=374)

Features to include:

In [278]:
#HOURLY LOG DATA: GOES BACK 6 HOURS
log_return = ['log_return_h'+str(x) for x in range(1,7)]
log_high = ['log_high_h'+str(x) for x in range(1,7)]
log_volume = ['log_volumefrom_h'+str(x) for x in range(1,7)]
#SUM DATA
sum_log_open = ["log_"+str(x)+"h_open" for x in [3,6,9,12,24,36] ]
sum_log_high = ["log_"+str(x)+"h_high" for x in [3,6,9,12,24,36] ]
sum_volumefrom = ['sum_'+str(x)+'h_volumefrom'for x in [3,6,9,12,24,36]]
sum_volumeto = ['sum_'+str(x)+'h_volumeto'for x in [3,6,9,12,24,36]]
base_feats = ['channelLink','channelTitle','currency','exchange','coin_name']
#MISC. DATA
time_data = ['pump_year','pump_month','pump_weekday','pump_hour']

In [279]:
o_features =  ['exchange_cat'] + time_data +  log_return + log_high + log_volume + sum_log_open + sum_log_high + sum_volumefrom + sum_volumeto

# BINANCE

In [280]:
#We will look at only Binance data
b = d[d.exchange == "Binance"]
print("There are {} data points in Binance".format(d.shape[0]))

There are 43506 data points in Binance


In [203]:
b[features + base_feats].head(3)

,exchange_cat,pump_year,pump_month,pump_weekday,pump_hour,log_return_h1,log_return_h2,log_return_h3,log_return_h4,log_return_h5,log_return_h6,log_high_h1,log_high_h2,log_high_h3,log_high_h4,log_high_h5,log_high_h6,log_volumefrom_h1,log_volumefrom_h2,log_volumefrom_h3,log_volumefrom_h4,log_volumefrom_h5,log_volumefrom_h6,log_3h_open,log_6h_open,log_9h_open,log_12h_open,log_24h_open,log_36h_open,log_3h_high,log_6h_high,log_9h_high,log_12h_high,log_24h_high,log_36h_high,sum_3h_volumefrom,sum_6h_volumefrom,sum_9h_volumefrom,sum_12h_volumefrom,sum_24h_volumefrom,sum_36h_volumefrom,sum_3h_volumeto,sum_6h_volumeto,sum_9h_volumeto,sum_12h_volumeto,sum_24h_volumeto,sum_36h_volumeto,channelLink,channelTitle,currency,exchange,coin_name
1620,0,2018,9,0,14,0.001509,0.008671,0.005517,0.004376,0.000000,0.005135,-0.042622,-0.055959,0.067536,-0.152277,0.020203,0.029776,-0.357811,-1.860519,-0.880844,2.098907,-0.461861,-1.424360,-0.032692,-0.118119,-0.253494,-0.153719,0.003026,-0.352740,-0.031045,-0.133344,-0.248642,-0.175794,0.006024,-0.349895,4.740495e+12,1.665632e+13,3.328419e+13,5.542504e+13,8.183836e+13,1.149128e+14,3.402400e+08,1.160540e+09,2.475540e+09,4.424540e+09,6.346490e+09,9.069600e+09,https://t.me/joinchat/AAAAAETRfXVAYuIieZ7K6g,Crypto Trading.,DNT,Binance,SALT
1621,0,2018,9,0,14,-0.008205,0.006651,0.008201,0.042803,0.008452,-0.003908,-0.070548,-0.049826,0.030153,-0.197207,0.023893,-0.064197,1.762726,-2.002445,-1.523415,3.129602,-0.828386,0.137518,-0.049811,-0.317618,-0.325763,-0.378898,-0.006110,-0.424687,-0.090221,-0.327732,-0.332840,-0.407831,-0.038830,-0.435796,2.987310e+13,1.114464e+14,1.648195e+14,2.004899e+14,3.908148e+14,4.708632e+14,3.113400e+08,1.179140e+09,1.919170e+09,2.431800e+09,4.891720e+09,5.875067e+09,https://t.me/joinchat/AAAAAETRfXVAYuIieZ7K6g,Crypto Trading.,DNT,Binance,POA
1622,0,2018,9,0,14,0.001362,0.002592,-0.017822,0.018955,0.000796,-0.006936,-0.087687,-0.137911,0.015811,-0.229498,-0.062904,0.089380,-0.963875,0.803721,0.187596,0.801292,-0.077188,-1.410568,-0.186032,-0.372290,-0.204381,-0.071831,0.394191,0.120202,-0.209787,-0.412809,-0.198065,-0.063194,0.401175,0.128839,2.131180e+13,2.967190e+13,4.500500e+13,4.961640e+13,1.333806e+14,2.165655e+14,1.916800e+09,2.858000e+09,4.589300e+09,5.043890e+09,1.101049e+10,1.624072e+10,https://t.me/joinchat/AAAAAETRfXVAYuIieZ7K6g,Crypto Trading.,DNT,Binance,MDA


## Spit Data
The __subsample__ function incoporates the sklearn.train_test_split_ function in the import section

In [11]:
x_train, x_test, y_train, y_test = subsample(b[o_features], b['pumped_yn'], num=15000)

Original :
x_train:(21844, 47), x_test:(5462, 47), y_train:(21844,), y_test:(5462,)
There are 200 positive samples in the original train set!
There are 46 positive samples in the original train set!
Subsample :
x_train:(15200, 47), y_train:(15200,), x_test:(5462, 47), y_test:(5462,)


## II. RANDOM FOREST

In [36]:
from sklearn.ensemble import RandomForestClassifier as RF

In [37]:
rf = RF()
params_rf = {'n_estimators':[50,100,200,500]
            ,'max_features': ['auto', 'sqrt', 'log2']
            ,'max_depth' : [10,15,25,50,75]
            ,'criterion' :['gini', 'entropy']}

In [38]:
CV_rfc = GridSearchCV(estimator=rf, param_grid=params_rf, cv=5, scoring='roc_auc')
CV_rfc.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [50, 100, 200, 500], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 15, 25, 50, 75], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [39]:
CV_rfc.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'log2',
 'n_estimators': 50}

In [12]:
rf_best = RF(criterion='entropy',max_depth=10,max_features='log2',n_estimators=500)
rf_best.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=10, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## ON TRAIN RESULT

In [13]:
y_train_prob = rf_best.predict_proba(x_train)[:,1]
# y_train_prob[y_train_prob > 0.16]

In [14]:
y_train_pred = np.zeros_like(y_train)
y_train_pred[y_train_prob > 0.1] = 1

In [15]:
# sum(y_train_pred)

In [16]:
tn,fp,fn,tp = confusion_matrix(y_train,y_train_pred).ravel()

In [17]:
print(tn,fp,fn,tp)

14968 32 36 164


## ON TEST RESULT

In [141]:
y_test_prob = rf_best.predict_proba(x_test)[:,1]

In [147]:
p = 0.1
y_test_pred = np.zeros_like(y_test)
y_test_pred[y_test_prob > p] = 1

In [148]:
tn,fp,fn,tp = confusion_matrix(y_test,y_test_pred).ravel()

In [149]:
print(tn,fp,fn,tp)

594 12 5 1


In [20]:
del x_train, y_train, x_test, y_test

# BITREX

In [126]:
#We will look at only Binance data
bt = d[d.exchange == "Bittrex"]
print("There are {} data points in Bittrex".format(d.shape[0]))

There are 43506 data points in Bittrex


In [129]:
x_train, x_test, y_train, y_test = subsample(bt[features], bt['pumped_yn'], num=2000)

Original :
x_train:(2448, 47), x_test:(612, 47), y_train:(2448,), y_test:(612,)
There are 28 positive samples in the original train set!
There are 6 positive samples in the original train set!
Subsample :
x_train:(2028, 47), y_train:(2028,), x_test:(612, 47), y_test:(612,)


In [ ]:
rf = RF()
params_rf = {'n_estimators':[50,100,200,500]
            ,'max_features': ['auto', 'sqrt', 'log2']
            ,'max_depth' : [10,15,25,50,75]
            ,'criterion' :['gini', 'entropy']}

In [ ]:
CV_rfc = GridSearchCV(estimator=rf, param_grid=params_rf, cv=5, scoring='roc_auc')
CV_rfc.fit(x_train, y_train)

In [ ]:
CV_rfc.best_params_

## YOBIT

# PART II. SOCIAL DATA

In [281]:
s = pd.read_pickle("../Log_processed_data/social_data_training_h_log.pkl")
print("Daily data",s.shape)

Daily data (80940, 377)


In [282]:
# Convert exchange variable to categorical
s['exchange_cat'] = s.exchange.astype('category').cat.codes

In [283]:
s.head(3)

,channelLink,channelTitle,currency,exchange,coin_name,timestamp,log_1h_analysis_page_views,log_3h_analysis_page_views,log_6h_analysis_page_views,log_9h_analysis_page_views,log_12h_analysis_page_views,log_24h_analysis_page_views,log_36h_analysis_page_views,log_1h_charts_page_views,log_3h_charts_page_views,log_6h_charts_page_views,log_9h_charts_page_views,log_12h_charts_page_views,log_24h_charts_page_views,log_36h_charts_page_views,log_1h_code_repo_closed_issues,log_3h_code_repo_closed_issues,log_6h_code_repo_closed_issues,log_9h_code_repo_closed_issues,log_12h_code_repo_closed_issues,log_24h_code_repo_closed_issues,log_36h_code_repo_closed_issues,log_1h_code_repo_closed_pull_issues,log_3h_code_repo_closed_pull_issues,log_6h_code_repo_closed_pull_issues,log_9h_code_repo_closed_pull_issues,log_12h_code_repo_closed_pull_issues,log_24h_code_repo_closed_pull_issues,log_36h_code_repo_closed_pull_issues,log_1h_code_repo_contributors,log_3h_code_repo_contributors,log_6h_code_repo_contributors,log_9h_code_repo_contributors,log_12h_code_repo_contributors,log_24h_code_repo_contributors,log_36h_code_repo_contributors,log_1h_code_repo_forks,log_3h_code_repo_forks,log_6h_code_repo_forks,log_9h_code_repo_forks,log_12h_code_repo_forks,log_24h_code_repo_forks,log_36h_code_repo_forks,log_1h_code_repo_open_issues,log_3h_code_repo_open_issues,log_6h_code_repo_open_issues,log_9h_code_repo_open_issues,log_12h_code_repo_open_issues,log_24h_code_repo_open_issues,log_36h_code_repo_open_issues,log_1h_code_repo_open_pull_issues,log_3h_code_repo_open_pull_issues,log_6h_code_repo_open_pull_issues,log_9h_code_repo_open_pull_issues,log_12h_code_repo_open_pull_issues,log_24h_code_repo_open_pull_issues,log_36h_code_repo_open_pull_issues,log_1h_code_repo_stars,log_3h_code_repo_stars,log_6h_code_repo_stars,log_9h_code_repo_stars,log_12h_code_repo_stars,log_24h_code_repo_stars,log_36h_code_repo_stars,log_1h_code_repo_subscribers,log_3h_code_repo_subscribers,log_6h_code_repo_subscribers,log_9h_code_repo_subscribers,log_12h_code_repo_subscribers,log_24h_code_repo_subscribers,log_36h_code_repo_subscribers,log_1h_comments,log_3h_comments,log_6h_comments,log_9h_comments,log_12h_comments,log_24h_comments,log_36h_comments,log_1h_fb_likes,log_3h_fb_likes,log_6h_fb_likes,log_9h_fb_likes,log_12h_fb_likes,log_24h_fb_likes,log_36h_fb_likes,log_1h_fb_talking_about,log_3h_fb_talking_about,log_6h_fb_talking_about,log_9h_fb_talking_about,log_12h_fb_talking_about,log_24h_fb_talking_about,log_36h_fb_talking_about,log_1h_followers,log_3h_followers,log_6h_followers,log_9h_followers,log_12h_followers,log_24h_followers,log_36h_followers,log_1h_forum_page_views,log_3h_forum_page_views,log_6h_forum_page_views,log_9h_forum_page_views,log_12h_forum_page_views,log_24h_forum_page_views,log_36h_forum_page_views,log_1h_influence_page_views,log_3h_influence_page_views,log_6h_influence_page_views,log_9h_influence_page_views,log_12h_influence_page_views,log_24h_influence_page_views,log_36h_influence_page_views,log_1h_markets_page_views,log_3h_markets_page_views,log_6h_markets_page_views,log_9h_markets_page_views,log_12h_markets_page_views,log_24h_markets_page_views,log_36h_markets_page_views,log_1h_overview_page_views,log_3h_overview_page_views,log_6h_overview_page_views,log_9h_overview_page_views,log_12h_overview_page_views,log_24h_overview_page_views,log_36h_overview_page_views,log_1h_points,log_3h_points,log_6h_points,log_9h_points,log_12h_points,log_24h_points,log_36h_points,log_1h_posts,log_3h_posts,log_6h_posts,log_9h_posts,log_12h_posts,log_24h_posts,log_36h_posts,log_1h_reddit_active_users,log_3h_reddit_active_users,log_6h_reddit_active_users,log_9h_reddit_active_users,log_12h_reddit_active_users,log_24h_reddit_active_users,log_36h_reddit_active_users,log_1h_reddit_comments_per_day,log_3h_reddit_comments_per_day,log_6h_reddit_comments_per_day,log_9h_reddit_comments_per_day,log_12h_reddit_comments_per_day,log_24h_reddit_comments_per_day,log_36h_reddit_comments_per_day,log_1h_reddit_commen

...All the variables are already curated...

In [284]:
base_feats = ['channelLink','channelTitle','currency','exchange','coin_name','timestamp']
s_features = list(set(s.columns) - set(base_feats))

In [285]:
s['pumped_yn'] = 0
s.loc[s.currency == s.coin_name,"pumped_yn"] = 1

## Binance

In [125]:
#We will look at only Binance data
b = s[s.exchange == "Binance"]
print("There are {} data points in Binance".format(d.shape[0]))

There are 43506 data points in Binance


In [126]:
b[s_features].head(3)

,log_1h_fb_talking_about,sum_6h_twitter_statuses,log_12h_fb_talking_about,log_24h_markets_page_views,log_36h_total_page_views,log_1h_influence_page_views,sum_3h_posts,sum_24h_fb_talking_about,log_12h_comments,log_12h_code_repo_forks,sum_12h_reddit_comments_per_day,log_3h_markets_page_views,log_3h_fb_likes,log_24h_reddit_subscribers,sum_1h_reddit_active_users,log_9h_twitter_statuses,sum_9h_code_repo_closed_pull_issues,log_12h_trades_page_views,log_12h_reddit_comments_per_hour,log_12h_reddit_subscribers,log_1h_analysis_page_views,sum_12h_code_repo_subscribers,sum_36h_code_repo_closed_pull_issues,log_6h_reddit_active_users,sum_36h_code_repo_open_issues,sum_1h_code_repo_open_issues,log_12h_code_repo_closed_issues,exchange_cat,log_3h_twitter_statuses,log_3h_posts,log_3h_code_repo_closed_issues,sum_9h_twitter_statuses,sum_3h_reddit_subscribers,sum_36h_twitter_followers,log_12h_twitter_favourites,log_1h_comments,log_36h_code_repo_closed_issues,log_1h_reddit_posts_per_day,sum_9h_reddit_posts_per_hour,log_36h_charts_page_views,log_6h_twitter_favourites,sum_3h_code_repo_open_pull_issues,log_12h_fb_likes,log_6h_code_repo_forks,log_6h_reddit_posts_per_day,log_9h_code_repo_subscribers,log_1h_markets_page_views,log_12h_twitter_statuses,sum_9h_reddit_posts_per_day,log_24h_reddit_comments_per_day,log_12h_reddit_posts_per_day,log_9h_comments,sum_6h_posts,log_36h_trades_page_views,log_6h_posts,sum_1h_twitter_following,log_6h_followers,sum_36h_code_repo_contributors,sum_9h_code_repo_contributors,log_24h_reddit_posts_per_hour,log_3h_code_repo_contributors,log_36h_reddit_comments_per_hour,log_9h_reddit_comments_per_hour,log_12h_reddit_active_users,log_6h_total_page_views,log_1h_twitter_following,sum_9h_code_repo_closed_issues,sum_6h_twitter_followers,log_6h_code_repo_closed_issues,log_24h_overview_page_views,sum_3h_twitter_statuses,log_12h_influence_page_views,log_6h_charts_page_views,sum_1h_twitter_favourites,sum_9h_posts,log_1h_code_repo_contributors,log_24h_forum_page_views,sum_1h_code_repo_open_pull_issues,log_9h_twitter_favourites,log_24h_fb_likes,log_12h_code_repo_stars,log_3h_reddit_posts_per_day,sum_24h_reddit_active_users,log_12h_twitter_followers,sum_1h_code_repo_stars,log_1h_fb_likes,log_6h_twitter_statuses,sum_9h_twitter_followers,sum_3h_fb_talking_about,log_12h_reddit_posts_per_hour,log_1h_twitter_lists,sum_12h_code_repo_closed_pull_issues,sum_24h_reddit_comments_per_day,log_24h_followers,log_24h_twitter_followers,sum_6h_twitter_favourites,sum_12h_code_repo_contributors,sum_24h_twitter_statuses,log_1h_code_repo_open_pull_issues,sum_24h_posts,log_12h_posts,log_6h_fb_talking_about,sum_6h_code_repo_subscribers,sum_36h_code_repo_subscribers,sum_24h_code_repo_contributors,sum_6h_reddit_posts_per_day,log_24h_reddit_comments_per_hour,sum_12h_posts,log_3h_forum_page_views,log_12h_markets_page_views,log_1h_reddit_active_users,sum_1h_twitter_lists,log_1h_posts,log_12h_code_repo_open_pull_issues,sum_24h_twitter_following,log_24h_twitter_lists,sum_9h_twitter_favourites,log_36h_comments,log_1h_followers,sum_36h_reddit_comments_per_day,sum_1h_twitter_followers,log_6h_comments,log_24h_code_repo_open_issues,log_36h_posts,sum_9h_code_repo_open_issues,sum_3h_code_repo_contributors,sum_1h_twitter_statuses,log_24h_code_repo_closed_issues,log_24h_points,sum_6h_twitter_following,log_12h_charts_page_views,log_3h_influence_page_views,log_6h_reddit_posts_per_hour,log_36h_followers,sum_9h_code_repo_stars,sum_24h_reddit_posts_per_day,log_36h_markets_page_views,sum_3h_reddit_posts_per_day,log_1h_charts_page_views,log_9h_posts,log_9h_reddit_comments_per_day,sum_12h_twitter_following,log_3h_code_repo_subscribers,sum_3h_code_repo_open_issues,log_1h_forum_page_views,log_6h_twitter_following,log_12h_forum_page_views,log_3h_code_repo_open_issues,log_24h_twitter_favourites,sum_36h_fb_talking_about,log_24h_code_repo_forks,sum_6h_code_repo_open_pull_issues,sum_24h_code_repo_subscribers,sum_9h_reddit_comments_per_hour,log_6h_code_repo_contributors,sum_12h_twitter_favourit

## Split Data

In [ ]:
x_train, x_test, y_train, y_test = subsample(b[features], b['pumped_yn'], num=15000)
rf = RF()
params_rf = {'n_estimators':[50,100,200,500]
            ,'max_features': ['auto', 'sqrt', 'log2']
            ,'max_depth' : [10,15,25,50,75]
            ,'criterion' :['gini', 'entropy']}
CV_rfc = GridSearchCV(estimator=rf, param_grid=params_rf, cv=5, scoring='roc_auc')
CV_rfc.fit(x_train, y_train)

In [23]:
# CV_rfc.best_params_

In [ ]:
# b['pumped_yn'].plot()

## ON TRAIN DATA


In [16]:
rf_best = RF(criterion='entropy',max_depth=10, max_features='auto',
            n_estimators=200)
rf_best.fit(x_train,y_train)
y_train_prob = rf_best.predict_proba(x_train)[:,1]
y_train_pred = np.zeros_like(y_train)
y_train_pred[y_train_prob > 0.1] = 1

In [18]:
tn,fp,fn,tp = confusion_matrix(y_train,y_train_pred).ravel()
print(tn,fp,fn,tp)

14980 20 48 149


## ON TEST DATA

In [21]:
y_test_prob = rf_best.predict_proba(x_test)[:,1]
y_test_pred = np.zeros_like(y_test)
y_test_pred[y_test_prob > 0.05] = 1

NameError: name 'x_test' is not defined

In [19]:
tn,fp,fn,tp = confusion_matrix(y_test,y_test_pred).ravel()
print(tn,fp,fn,tp)

5347 69 43 3


In [ ]:
del x_train, x_test, y_train, y_test

# PART III. COMBINE BOTH SOCIAL AND OHLC DATA

In [341]:
merge_vars = ['channelTitle','currency','exchange','coin_name','timestamp']
f = pd.merge(left=d,left_on=merge_vars,right=s,right_on=merge_vars,how='inner')
f['exchange_cat'] = f.exchange.astype('category').cat.codes
f.shape

(43472, 1057)

In [342]:
#Combine features
f_features = np.unique(list(set(o_features + s_features) - set(['pumped_yn'])))
print("Number of features",len(f_features))

Number of features 418


In [343]:
f["pumped_yn"] = 0
f.loc[f.currency == f.coin_name, "pumped_yn"] = 1

In [345]:
## Binance
f = f.loc[f.exchange == 'Binance']
#Split
x_train, x_test, y_train, y_test = subsample(f[f_features], f['pumped_yn'], num=10000)

Original :
x_train:(21844, 418), x_test:(5462, 418), y_train:(21844,), y_test:(5462,)
There are 204 positive samples in the original train set!
There are 42 positive samples in the original train set!
Subsample :
x_train:(10204, 418), y_train:(10204,), x_test:(5462, 418), y_test:(5462,)


In [300]:
rf_best = RF(criterion='entropy',max_depth=10,max_features='log2',n_estimators=500)
rf_best.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=10, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [304]:
#Train data
y_train_prob = rf_best.predict_proba(x_train)[:,1]
y_train_pred = np.zeros_like(y_train)
y_train_pred[y_train_prob > 0.2] = 1
getConfusionMatrix(y_train,y_train_pred)

TN 29996 
FP 4 
FN 296 
TP 42


(29996, 4, 296, 42)

In [308]:
#Test
y_test_prob = rf_best.predict_proba(x_test)[:,1]
y_test_pred = np.zeros_like(y_test)
y_test_pred[y_test_prob > 0.1] = 1
getConfusionMatrix(y_test,y_test_pred)

TN 8594 
FP 13 
FN 85 
TP 3


(8594, 13, 85, 3)

## SMOTE

In [347]:
from imblearn.over_sampling import SMOTE 

In [348]:
sm = SMOTE(random_state=123)
x_train_sm,y_train_sm = sm.fit_sample(x_train, y_train)

In [349]:
#Train data
rf_best = RF(criterion='entropy',max_depth=10,max_features='log2',n_estimators=500)
rf_best.fit(x_train_sm, y_train_sm)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=10, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [350]:
#Train
y_train_prob = rf_best.predict_proba(x_train_sm)[:,1]
y_train_pred = np.zeros_like(y_train_sm)
y_train_pred[y_train_prob > 0.55] = 1
getConfusionMatrix(y_train_sm, y_train_pred)

TN 9677 
FP 323 
FN 319 
TP 9681


(9677, 323, 319, 9681)

In [352]:
#Test
y_test_prob = rf_best.predict_proba(x_test)[:,1]
y_test_pred = np.zeros_like(y_test)
y_test_pred[y_test_prob > 0.55] = 1
getConfusionMatrix(y_test,y_test_pred)

TN 5165 
FP 255 
FN 36 
TP 6


(5165, 255, 36, 6)

## ADASYN

In [353]:
from imblearn.over_sampling import ADASYN

In [354]:
ads = ADASYN(random_state=123)
x_train_ads, y_train_ads = ads.fit_sample(x_train, y_train)

In [355]:
#Train data
rf_best = RF(criterion='entropy',max_depth=10,max_features='log2',n_estimators=500)
rf_best.fit(x_train_ads, y_train_ads)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=10, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [333]:
#Train
y_train_prob = rf_best.predict_proba(x_train_sm)[:,1]
y_train_pred = np.zeros_like(y_train_sm)
y_train_pred[y_train_prob > 0.5] = 1
getConfusionMatrix(y_train_sm, y_train_pred)

TN 29074 
FP 926 
FN 2347 
TP 27653


(29074, 926, 2347, 27653)

In [357]:
#Test
y_test_prob = rf_best.predict_proba(x_test)[:,1]
y_test_pred = np.zeros_like(y_test)
y_test_pred[y_test_prob > 0.5] = 1
getConfusionMatrix(y_test,y_test_pred)

TN 5010 
FP 410 
FN 34 
TP 8


(5010, 410, 34, 8)

In [358]:
f_features

array(['exchange_cat', 'log_12h_analysis_page_views',
       'log_12h_charts_page_views', 'log_12h_code_repo_closed_issues',
       'log_12h_code_repo_closed_pull_issues',
       'log_12h_code_repo_contributors', 'log_12h_code_repo_forks',
       'log_12h_code_repo_open_issues',
       'log_12h_code_repo_open_pull_issues', 'log_12h_code_repo_stars',
       'log_12h_code_repo_subscribers', 'log_12h_comments',
       'log_12h_fb_likes', 'log_12h_fb_talking_about',
       'log_12h_followers', 'log_12h_forum_page_views', 'log_12h_high',
       'log_12h_influence_page_views', 'log_12h_markets_page_views',
       'log_12h_open', 'log_12h_overview_page_views', 'log_12h_points',
       'log_12h_posts', 'log_12h_reddit_active_users',
       'log_12h_reddit_comments_per_day',
       'log_12h_reddit_comments_per_hour', 'log_12h_reddit_posts_per_day',
       'log_12h_reddit_posts_per_hour', 'log_12h_reddit_subscribers',
       'log_12h_total_page_views', 'log_12h_trades_page_views',
       'log_12